# MODEL VERIFICATION

When creating the model from scratch and manipulating the data in different ways to adapt it to the use case, verifying the model at each step turns out to be important.

During LAB2 notebook, we already verified the FINN-ONNX model, which indicates that we did the job right on our side, but what if something goes wrong in the way we apply further transformations ?

To make sure we are ready for FPGA inference, verification is a very important step to avoid hours of useless hardware dubugging.

Verifications convered by this notebook :

- HLS layers verification using C++
- RTL output verification using PyVerilator

This notebook was based on [this example](https://github.com/Xilinx/finn/blob/main/notebooks/end2end_example/bnn-pynq/tfc_end2end_verification.ipynb) from FINN tutorials.

As you will see, verification will be fairly easy as FINN provides a very user-friendly API for these tools.

## Workflow : manual transformations

In this notebook, we will use [fpgadataflow transformations](https://finn.readthedocs.io/en/latest/source_code/finn.transformation.fpgadataflow.html) manualy. These usually are done automatically when building in FINN notebooks, depending on what output you ask for. Because we want control in these simulation examples, we wil use transformations "manualy".

In [1]:
root_dir = "/tmp/finn_dev_rootmin"

# C++ Simulation

First, execute LAB2, we will grab the models from the common ```/tmp/finn_dev_yourusername/``` output folder

We first define the "golden reference" for comparison

In [ ]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

input_tensor = input_a = np.random.uniform(low=0, high=255, size=(28*28)).astype(np.uint8).astype(np.float32)
input_dict = {"global_in": input_tensor.reshape(1,28*28)}
golden_model = ModelWrapper(root_dir + "/full_preproc.onnx")
output_dict = oxe.execute_onnx(golden_model, input_dict)
golden_output = output_dict[list(output_dict.keys())[0]]

print(golden_output)

We will generate the different source code : ```PrepareCppSim``` and executables : ```CompileCppSim```

In [ ]:
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from qonnx.transformation.general import GiveUniqueNodeNames
from qonnx.core.modelwrapper import ModelWrapper

model_cppsim = ModelWrapper(root_dir + "/to_hw_conv.onnx")
model_cppsim = model_cppsim.transform(GiveUniqueNodeNames())
model_cppsim = model_cppsim.transform(PrepareCppSim())
model_cppsim = model_cppsim.transform(CompileCppSim())

from finn.util.visualization import showSrc, showInNetron

model_cppsim.save(root_dir + "/cppsim.onnx")
showInNetron(root_dir + "/cppsim.onnx")


graph manipulation reminder : [cutomOp Docs](https://finn.readthedocs.io/en/latest/source_code/finn.custom_op.html#module-qonnx.custom_op.registry)

In [ ]:
# Look at the generated files
from qonnx.custom_op.registry import getCustomOp

model = ModelWrapper(root_dir + "/cppsim.onnx")

fc0 = model.graph.node[0]
fc0w = getCustomOp(fc0)
cpp_code_dir = fc0w.get_nodeattr("code_gen_dir_cppsim")

!ls {cpp_code_dir}

## Simulation and testing

In [5]:
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model_cppsim = model_cppsim.transform(SetExecMode("cppsim"))
model_cppsim.save(root_dir + "/cppsim_exec.onnx")

In [ ]:
import numpy as np
import onnx.numpy_helper as nph
import qonnx.core.onnx_exec as oxe

input_dict = {"global_in": input_tensor.reshape(1,28*28)}

parent_model = ModelWrapper(root_dir + "/df_part.onnx")
sdp_node = parent_model.graph.node[0]
child_model = root_dir + "/cppsim_exec.onnx"
getCustomOp(sdp_node).set_nodeattr("model", child_model)
output_dict = oxe.execute_onnx(parent_model, input_dict)
output_cppsim = output_dict[list(output_dict.keys())[0]]

try:
    print(golden_output[0], output_cppsim[0])
    assert np.isclose(output_cppsim[0], np.where(golden_output[0]==np.amax(golden_output[0])), atol=1e-3).all()
    print("Predictions are the same!")
except AssertionError:
    assert False, "The results are not the same!"

Great results are the same ! Note that this very small exmaple was done as an example and compares simple top label output. You can use the exmaple in a loop to check for hundreds of random sample et even setup a dataloader and testing loop for verification like we did like in LAB2.

# PyVerilator RTL Verification / Emulation

Once the RTL has been generated, we can also emulate it to compare with the golden result. They are multpile ways to go about this simulation, we will go for a "node by node" method.

manual worflow comm

In [7]:
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveUniqueNodeNames


test_fpga_part = "xc7z020clg400-1"
target_clk_ns = 10

In [ ]:
from finn.util.visualization import showSrc, showInNetron

model_rtlsim = ModelWrapper(root_dir + "/post_synth.onnx")
showInNetron(root_dir + "/post_synth.onnx")

In [9]:
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model_rtlsim = model_rtlsim.transform(SetExecMode("rtlsim"))
model_rtlsim = model_rtlsim.transform(PrepareRTLSim())
model_rtlsim.save(root_dir + "/rtlsim.onnx")

In [10]:
# declare parent model from dataflow partition parent model
parent_rtsim = ModelWrapper(root_dir + "/df_part.onnx")
# reference child rtl model to the streming dataflow node
sdp_node = getCustomOp(parent_model.graph.node[0])
sdp_node.set_nodeattr("model", root_dir + "/rtlsim.onnx")

# set the exec mode for when we'll use oxe runtime, just like with C++ simulation
parent_rtsim = parent_rtsim.transform(SetExecMode("rtlsim"))

In [ ]:
#declare input data and run oxe runtime inference in RTL SIM mode
input_dict = {"global_in": input_tensor.reshape(1,28*28)}
output_dict = oxe.execute_onnx(parent_rtsim, input_dict)
output_rtlsim = output_dict[list(output_dict.keys())[0]]
print(golden_output, output_rtlsim)

In [ ]:
try:
    assert np.isclose(output_rtlsim, np.where(golden_output[0]==np.amax(golden_output[0])), atol=1e-3).all()
    print("Predictions are the same!")
except AssertionError:
    assert False, "The results are not the same!"